In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from copy import deepcopy

import k3d
import yaml
import numpy as np

import matplotlib.pyplot as plt

from tqdm import tqdm
import json

import glob
import os
import trimesh.transformations as tt


Bad key "text.kerning_factor" on line 4 in
/home/user/miniconda/envs/py36/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [7]:
import meshplot as mp 

In [8]:
def display_sharpness(mesh=None, plot_meshvert=True,
                      plot_mesh_wireframe=False, mesh_wireframe_color=0x000000,
                      samples=None, samples_distances=None,
                      sharp_vert=None, sharp_curves=None,
                      directions=None, directions_width=0.0025,
                      samples_color=0x0000ff, samples_psize=0.002, 
                      mesh_color=0xbbbbbb, meshvert_color=0x666666, meshvert_psize=0.0025,
                      sharpvert_color=0xff0000, sharpvert_psize=0.0025,
                      sharpcurve_color=None, sharpcurve_width=0.0025,
                      as_image=False, plot_height=768, camera_fov=60.0, lighting=1.5):
    
    plot = k3d.plot(
        height=plot_height,
        camera_fov=camera_fov,
        lighting=lighting, 
        camera_auto_fit=False,
        grid_visible=False)
    
    if None is not mesh:
        k3d_mesh = k3d.mesh(
            mesh.vertices,
            mesh.faces,
            color=mesh_color)
        plot += k3d_mesh

        if plot_meshvert:
            k3d_points = k3d.points(
                mesh.vertices, 
                point_size=meshvert_psize,
                color=meshvert_color)
            plot += k3d_points
            k3d_points.shader='3d'
        
        if plot_mesh_wireframe:
            k3d_mesh = k3d.mesh(
                mesh.vertices,
                mesh.faces,
                wireframe=True,
                color=mesh_wireframe_color)
            plot += k3d_mesh

    if None is not samples:
        colors = None
        if None is not samples_distances:
            max_dist = np.max(samples_distances)

            colors = k3d.helpers.map_colors(
                samples_distances, k3d.colormaps.basic_color_maps.WarmCool, [0, max_dist]
            ).astype(np.uint32)
            k3d_points = k3d.points(samples, point_size=samples_psize, colors=colors)
        else:
            k3d_points = k3d.points(samples, point_size=samples_psize, color=samples_color)
        plot += k3d_points
        k3d_points.shader='flat'
        
        if None is not directions:
            vectors = k3d.vectors(samples, directions * samples_distances[..., np.newaxis])
            plot += vectors

#             directions_to_plot = np.hstack((samples, samples + directions))
            
#             for i, dir_to_plot in enumerate(directions_to_plot):
#                 dir_to_plot = dir_to_plot.reshape((2, 3))
#                 if np.all(dir_to_plot[0] == dir_to_plot[1]):
#                     continue
#                 color = int(colors[i]) if None is not colors else samples_color
#                 plt_line = k3d.line(dir_to_plot, 
#                                     shader='mesh', width=directions_width, color=color)
#                 plot += plt_line

    if None is not sharp_vert:
        k3d_points = k3d.points(sharp_vert,
                                point_size=sharpvert_psize, color=sharpvert_color)
        plot += k3d_points
        k3d_points.shader='flat'
        
    if None is not sharp_curves:            
        if None is not sharpcurve_color:
            color = sharpcurve_color
        else:
            import randomcolor
            rand_color = randomcolor.RandomColor()
        for i, vert_ind in enumerate(sharp_curves):
            sharp_points_curve = mesh.vertices[vert_ind]

            if None is sharpcurve_color:
                color = rand_color.generate(hue='red')[0]
                color = int('0x' + color[1:], 16)
            plt_line = k3d.line(sharp_points_curve, 
                                shader='flat', 
                                width=sharpcurve_width, 
                                color=color)
            plot += plt_line
        
    plot.camera_fov = camera_fov
    plot.display()
    
    if as_image:
        plot.fetch_screenshot()
        return Image(data=b64decode(plot.screenshot))

In [68]:
def brep_like_rendering(mesh, sharp_curves):
    
    shading = {
        "flat":True,
        "wireframe":False,
        "scale":4.0
    }
    c = np.array([118., 178., 146.]) / 255
    p = mp.plot(mesh.vertices, mesh.faces, 
                c=c, shading=shading)

#     for i, vert_ind in enumerate(sharp_curves):
        
#         sharp_points_curve = mesh.vertices[vert_ind]

#     for _, fe in enumerate(feat["curves"]):
#         for j in range(len(fe["vert_indices"])-1):
#             lines.append([fe["vert_indices"][j], fe["vert_indices"][j+1]])

#     # Visualize the sharp features            
#     p = mp.plot(v, f)
#     p.add_edges(v, np.array(lines))
    
#     display_sharpness(
#         mesh, plot_meshvert=False, meshvert_psize=sampler.resolution_3d, mesh_color=0x76d8ad,
#         sharp_vert=None, sharpvert_psize=2. * sampler.resolution_3d,
#         samples=None, samples_psize=1. * sampler.resolution_3d,
#         sharp_curves=sharp_curves, sharpcurve_color=0x000000, sharpcurve_width=0.01,
#         camera_fov=15., lighting=0.5
#     )

In [10]:
# mm/pixel
HIGH_RES = 0.02
MED_RES = 0.05
LOW_RES = 0.125
XLOW_RES = 0.25

## 1. Working with point patches

In [11]:
from sharpf.utils.abc_utils.abc.abc_data import ABCModality, ABCChunk, ABC_7Z_FILEMASK, ABCItem
from sharpf.data.annotation import ANNOTATOR_BY_TYPE
from sharpf.data.mesh_nbhoods import NBHOOD_BY_TYPE
from sharpf.data.noisers import NOISE_BY_TYPE
from sharpf.data.point_samplers import SAMPLER_BY_TYPE
from sharpf.utils.py_utils.console import eprint
# from sharpf.utils.abc_utils.mesh.io import trimesh_load

from sharpf.utils.abc_utils.abc.feature_utils import (
    get_adjacent_features_by_bfs_with_depth1,
    build_surface_patch_graph,
    get_curves_extents,
    compute_features_nbhood,
    remove_boundary_features)
from sharpf.utils.geometry import dist_vector_proj, mean_mmd
from sharpf.utils.py_utils.config import load_func_from_config


import sharpf.utils.camera_utils.camera_pose as c

In [12]:
# data_holder = ABCChunk(['/data/abc_0056_obj_v00.7z', '/data/abc_0056_feat_v00.7z'])


In [13]:
# item = data_holder[584]
# item.item_id

In [14]:
from io import BytesIO

import trimesh
import numpy as np


def trimesh_load(io: BytesIO) -> trimesh.base.Trimesh:
    """Read the mesh: since trimesh messes the indices, this has to be done manually."""

    vertices, faces = [], []

    for line in io.read().splitlines():
        values = line.strip().split()
        if not values:
            continue
        if values[0] == 'v':
            vertices.append(np.array(values[1:4], dtype='float'))
        elif values[0] == 'f':
            faces.append(np.array([values[1].split('//')[0], values[2].split('//')[0], values[3].split('//')[0]],
                                  dtype='int'))

    vertices = np.array(vertices)
    faces = np.array(faces) - 1

    mesh = trimesh.base.Trimesh(vertices=vertices, faces=faces, process=False)  # create a mesh from the vertices
    return mesh



In [15]:
def get_item_by_id(base_path_list, item_id):
    item_prefix = item_id.split('_')[0]
    
    obj_dir, feat_dir = base_path_list
    
    item_obj_filename = os.path.join(
        obj_dir,
        '{}_trimesh_*.obj'.format(item_id)
    )
    item_obj_filename = glob.glob(item_obj_filename)[0]
    
    item_feat_filename = os.path.join(
        feat_dir,
        '{}_features_*.yml'.format(item_id)
    )
    item_feat_filename = glob.glob(item_feat_filename)[0]

    with open(item_obj_filename) as item_obj:
        mesh = trimesh_load(item_obj)
        
    with open(item_feat_filename) as item_feat:
        features = yaml.load(item_feat, Loader=yaml.Loader)

    return ABCItem(obj=mesh, feat=features)

In [26]:
with open('/code/scripts/data_scripts/configs/pointcloud_datasets/dataset_config_low_res_clean.json') as config_file:
    config = json.load(config_file)

config

{'shape_fabrication_extent': 10.0,
 'short_curve_quantile': 0.25,
 'base_n_points_per_short_curve': 8,
 'base_resolution_3d': 0.125,
 'neighbourhood': {'type': 'random_euclidean_sphere',
  'max_patches_per_mesh': 48,
  'n_vertices': None,
  'centroid': None,
  'centroid_mode': 'poisson_disk',
  'radius_base': 10.0,
  'radius_delta': 0.0,
  'geodesic_patches': True,
  'radius_scale_mode': 'no_scale'},
 'sampling': {'type': 'poisson_disk',
  'n_points': 4096,
  'resolution_3d': 0.125,
  'crop_center': True,
  'resolution_deviation_tolerance': 0.03},
 'noise': {'type': 'many_noisers',
  'subtype': 'isotropic_gaussian',
  'scale': [0.0]},
 'annotation': {'type': 'surface_based_aabb',
  'distance_upper_bound': 1.0,
  'validate_annotation': True}}

In [27]:
shape_fabrication_extent = config.get('shape_fabrication_extent', 10.0)
base_n_points_per_short_curve = config.get('base_n_points_per_short_curve', 8)
base_resolution_3d = config.get('base_resolution_3d', LOW_RES)

short_curve_quantile = config.get('short_curve_quantile', 0.05)

nbhood_extractor = load_func_from_config(NBHOOD_BY_TYPE, config['neighbourhood'])
sampler = load_func_from_config(SAMPLER_BY_TYPE, config['sampling'])
noiser = load_func_from_config(NOISE_BY_TYPE, config['noise'])
annotator = load_func_from_config(ANNOTATOR_BY_TYPE, config['annotation'])

# Specific to this script only: override radius of neighbourhood extractor
# to reflect actual point cloud resolution:
# we extract spheres of radius r, such that area of a (plane) disk with radius r
# is equal to the total area of 3d points (as if we scanned a plane wall)
nbhood_extractor.radius_base = np.sqrt(sampler.n_points) * 0.5 * sampler.resolution_3d

In [28]:

def scale_mesh(mesh, features, shape_fabrication_extent, resolution_3d,
               short_curve_quantile=0.05, n_points_per_short_curve=4):
    # compute standard size spatial extent
    mesh_extent = np.max(mesh.bounding_box.extents)
    mesh = mesh.apply_scale(shape_fabrication_extent / mesh_extent)

    # compute lengths of curves
    sharp_curves_lengths = get_curves_extents(mesh, features)

    least_len = np.quantile(sharp_curves_lengths, short_curve_quantile)
    least_len_mm = resolution_3d * n_points_per_short_curve

    mesh = mesh.apply_scale(least_len_mm / least_len)

    return mesh


In [29]:
# mesh = scale_mesh(mesh, features, shape_fabrication_extent, base_resolution_3d,
#                   short_curve_quantile=short_curve_quantile,
#                   n_points_per_short_curve=base_n_points_per_short_curve)

In [34]:
# item_id = '00009684_f3f3f0459fff4bcb8d7ab694'

item_id = '00006991_ceb6d89499b14c2d8a829e37'

item = get_item_by_id(
    ['/data/abc/abc_0000_obj_v00', '/data/abc/abc_0000_feat_v00'], 
    item_id)

mesh = item.obj
features = item.feat

In [42]:
with open('/data/abc/test_trimesh.obj') as item_obj:
    mesh = trimesh_load(item_obj)

with open('/data/abc/test_features.yml') as item_feat:
    features = yaml.load(item_feat, Loader=yaml.Loader)
    
mesh_orig = mesh.copy()

In [43]:
sharp_curves = [curve['vert_indices'] for curve in features['curves'] if curve['sharp']]

In [71]:
mesh = mesh_orig.copy()
mesh_extent = np.max(mesh.bounding_box.extents)
mesh = mesh.apply_scale(7.5 / mesh_extent)

# [z x y]
def _n(a):
    a = np.array(a)
    return a / np.linalg.norm(a)


R = tt.rotation_matrix(0*np.pi/180, [0., 1., 0.], np.mean(mesh.vertices, axis=0))
mesh.apply_transform(R)

# R = tt.rotation_matrix(45*np.pi/180, _n([1., 0., 0.]), np.mean(mesh.vertices, axis=0))
# mesh.apply_transform(R)

# R = tt.rotation_matrix(-75*np.pi/180, _n([0., -1., 1.]), np.mean(mesh.vertices, axis=0))
# mesh.apply_transform(R)

brep_like_rendering(mesh, sharp_curves=sharp_curves)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.7438485…

In [417]:
display_sharpness(
    mesh, plot_meshvert=False, meshvert_psize=sampler.resolution_3d, mesh_color=0x76d8ad,
    plot_mesh_wireframe=True,
    sharp_vert=None, sharpvert_psize=2. * sampler.resolution_3d,
    samples=None, samples_psize=1. * sampler.resolution_3d,
    sharp_curves=None, sharpcurve_color=0x000000, sharpcurve_width=0.01,
    camera_fov=15., lighting=0.5
)

Output()

In [20]:
# index the mesh using a neighbourhood functions class
# (this internally may call indexing, so for repeated invocation one passes the mesh)
nbhood_extractor.index(mesh)

# extract neighbourhood
try:
    nbhood, mesh_vertex_indexes, mesh_face_indexes, scaler = nbhood_extractor.get_nbhood()
except DataGenerationException as e:
    eprint(str(e))

display_sharpness(nbhood, plot_meshvert=True, meshvert_psize=sampler.resolution_3d/2)

Output()

In [40]:

display_sharpness(
    mesh, plot_meshvert=True, meshvert_psize=sampler.resolution_3d,
    sharp_vert=None, sharpvert_psize=2. * sampler.resolution_3d,
    samples=nbhood_extractor.centroids_cache, samples_psize=5. * sampler.resolution_3d,
)



Output()

In [34]:
nbhood.faces

TrackedArray([[145, 156,   6],
              [ 35, 124,  34],
              [ 27, 194, 154],
              ...,
              [728, 738, 723],
              [724, 740, 729],
              [733, 734, 725]])

In [21]:
# create annotations: condition the features onto the nbhood
nbhood_features = compute_features_nbhood(mesh, features, mesh_vertex_indexes, mesh_face_indexes)

# remove vertices lying on the boundary (sharp edges found in 1 face only)
nbhood_features = remove_boundary_features(nbhood, nbhood_features, how='edges')

In [22]:
if any(c['sharp'] for c in nbhood_features['curves']):
    sharp_verts = np.concatenate([nbhood.vertices[c['vert_indices']] 
                                  for c in nbhood_features['curves'] if c['sharp']])
else:
    sharp_verts = []
    

surf_vert_indices = np.array(nbhood_features['surfaces'][2]['vert_indices'])
surf_verts = nbhood.vertices[surf_vert_indices]



display_sharpness(
    nbhood, plot_meshvert=True, meshvert_psize=sampler.resolution_3d,
    sharp_vert=sharp_verts, sharpvert_psize=1. * sampler.resolution_3d,
    samples=surf_verts, samples_psize=sampler.resolution_3d,
)

Output()

In [23]:
len(nbhood.vertices)

742

In [29]:
nbhood.edges_unique

array([[  0,   4],
       [  4,   5],
       [  5,   6],
       ...,
       [ 78, 741],
       [726, 741],
       [730, 741]])

In [41]:
import igl

AttributeError: module 'igl' has no attribute '__version__'

In [24]:
sampler.resolution_deviation_tolerance = 1.0

In [25]:
# sample the neighbourhood to form a point patch
try:
    points, normals = sampler.sample(nbhood, centroid=nbhood_extractor.centroid)
except DataGenerationException as e:
    eprint(str(e))
    pass

display_sharpness(nbhood, plot_meshvert=False, meshvert_psize=sampler.resolution_3d / 10,
                  samples=points, samples_distances=None,
                  samples_color=0x0000ff, samples_psize=sampler.resolution_3d,
                  sharp_vert=sharp_verts, sharpvert_psize=1. * sampler.resolution_3d,)

Output()

In [250]:
i = iter(noiser.make_noise(points, normals))

In [251]:
# create a noisy sample
c, noisy_points = next(i)

display_sharpness(nbhood, plot_meshvert=True, meshvert_psize=sampler.resolution_3d/2,
                  samples=noisy_points, samples_distances=None,
                  samples_color=0x0000ff, samples_psize=sampler.resolution_3d,
                  sharp_vert=sharp_verts, sharpvert_psize=2*sampler.resolution_3d,)

Output()

In [91]:
annotator.validate_annotation=True

In [95]:
annotator.distance_upper_bound = 1.0
# compute the TSharpDF
distances, directions, has_sharp = annotator.annotate(nbhood, nbhood_features, noisy_points)



Output()

In [252]:
display_sharpness(nbhood, plot_meshvert=False, meshvert_psize=0.01,
                  sharp_vert=sharp_adjacent_2, sharpvert_psize=2 * sampler.resolution_3d,
                  samples=noisy_points, samples_distances=distances,
                  samples_color=0x0000ff, samples_psize=sampler.resolution_3d,
                  directions=directions)

Vectors(colors=array([], dtype=uint32), head_color=255, id=140691902224424, model_matrix=array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32), origin_color=255, origins=array([[-19.547358  ,  -7.849757  ,   0.946174  ],
       [-19.532969  ,  -8.027769  ,   0.83327705],
       [-19.54474   ,  -7.933261  ,   1.0442309 ],
       ...,
       [-16.622044  , -10.077516  ,   2.101587  ],
       [-20.155031  ,  -6.303346  ,   4.3024707 ],
       [-17.301134  , -10.077516  ,   3.136637  ]], dtype=float32), type='Vectors', use_head=True, vectors=array([[ 4.0007704e-01, -2.1225446e-01, -2.8303150e-08],
       [ 3.8568768e-01, -3.4243196e-02, -2.8609259e-08],
       [ 3.9745992e-01, -1.2875119e-01, -5.5509172e-08],
       ...,
       [-7.9386814e-07,  0.0000000e+00, -5.8995873e-01],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00]], dtype=float32))


Output()

In [233]:
adjacent_sharp_features, adjacent_surfaces = build_surface_patch_graph(nbhood_features)


In [253]:
surface_idx = 17
surface = nbhood_features['surfaces'][surface_idx]
adjacent_sharp_indexes = get_adjacent_features_by_bfs_with_depth1(
                surface_idx, adjacent_sharp_features, adjacent_surfaces)
surface_adjacent_features = {
    'curves': [nbhood_features['curves'][idx]
               for idx in np.unique(adjacent_sharp_indexes)]
}


In [258]:
# sharp_adjacent_2 = np.concatenate([nbhood.vertices[
#     nbhood_features['curves'][sharp_curve_id]['vert_indices']
# ] for sharp_curve_id in adjacent_sharp_features[2]])
sharp_adjacent_2 = np.concatenate([nbhood.vertices[
    nbhood_features['curves'][sharp_curve_id]['vert_indices']
] for sharp_curve_id in adjacent_sharp_features[17]])

sharp_adjacent_2_with_bfs = np.concatenate([nbhood.vertices[
    curve['vert_indices']
] for curve in surface_adjacent_features['curves']])

In [259]:
display_sharpness(None,
                 sharp_vert = sharp_adjacent_2, sharpvert_psize=0.05,
                 samples=sharp_adjacent_2_with_bfs, samples_psize=0.05,)

Output()

In [260]:
from sharpf.utils.mesh_utils.indexing import reindex_zerobased, compute_relative_indexes

In [261]:
surface = nbhood_features['surfaces'][surface_idx]

surface_face_indexes = np.where(np.isin(nbhood.faces, surface['vert_indices']).all(axis=1))[0]

surface_mesh = reindex_zerobased(nbhood, surface['vert_indices'], surface_face_indexes)

In [263]:

display_sharpness(surface_mesh,
                 sharp_vert = sharp_adjacent_2_with_bfs, sharpvert_psize=0.05,
                 samples=None, samples_psize=0.05,)

/home/user/miniconda/envs/py36/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))
/home/user/miniconda/envs/py36/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "int64" does not match required type "uint32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

In [167]:

def remove_boundary_features(mesh, features, how='none'):
    """Removes features indexed into vertex edges adjacent to 1 face only.
    :param how: 'all_verts': remove entire feature curve if all vertices are boundary
                'edges': remove vertices that belong to boundary edges only (not to other edges)
                'verts': remove vertices that are boundary
                'none': do nothing
    """
    if how == 'none':
        return features

    mesh_edge_indexes, mesh_edge_counts = np.unique(
        mesh.faces_unique_edges.flatten(), return_counts=True)

    boundary_edges = mesh.edges_unique[mesh_edge_indexes[np.where(mesh_edge_counts == 1)[0]]]
    boundary_vertex_indexes = np.unique(boundary_edges.flatten())

    non_boundary_curves = []
    for curve in features['curves']:
        non_boundary_curve = deepcopy(curve)

        if how == 'all_verts':
            if np.all([vert_index in boundary_vertex_indexes
                       for vert_index in curve['vert_indices']]):
                continue

        elif how == 'verts':
            non_boundary_vert_indices = np.array([
                vert_index for vert_index in curve['vert_indices']
                if vert_index not in boundary_vertex_indexes
            ])
            if len(non_boundary_vert_indices) == 0:
                continue
            non_boundary_curve['vert_indices'] = non_boundary_vert_indices

        elif how == 'edges':
            curve_edges = mesh.edges_unique[
                np.where(
                    np.all(np.isin(mesh.edges_unique, curve['vert_indices']), axis=1)
                )[0]
            ]
            non_boundary = (curve_edges[:, None] != boundary_edges).any(2).all(1)
            non_boundary_vert_indices = np.unique(curve_edges[non_boundary])
            non_boundary_curve['vert_indices'] = non_boundary_vert_indices

        non_boundary_curves.append(non_boundary_curve)

    non_boundary_features = {
        'curves': non_boundary_curves,
        'surfaces': features.get('surfaces', [])
    }
    return non_boundary_features


In [178]:
surface_adjacent_features = remove_boundary_features(surface_mesh, surface_adjacent_features, how='verts')


In [197]:
display_sharpness(surface_mesh,
                 sharp_vert = sharp_adjacent_2, sharpvert_psize=0.2)

Output()

In [49]:
from scipy.spatial import cKDTree


In [50]:
nn_distances, nn_indexes = cKDTree(noisy_points, leafsize=16).query(noisy_points, k=2, n_jobs=4)


In [51]:
distances[nn_indexes[:, 0]].shape

(4096,)

In [53]:
values = np.abs(distances[nn_indexes[:, 0]] - distances[nn_indexes[:, 1]]) / nn_distances[:, 1]

In [56]:
values[np.where(values > 1.1)[0]]

array([4.66502749, 5.17381566, 4.29546884, 4.66502749, 4.06572867,
       4.06572867, 3.17576882, 3.73807832, 2.26651807, 2.85688188,
       4.74473641, 4.83730064, 4.83596246, 4.83730064, 4.83596246,
       4.03023284, 5.14030622, 5.05465149, 3.58716084, 3.58716084,
       4.75233872, 4.79312687, 4.20335957, 4.38694014, 4.01801899,
       4.0858341 , 4.57970763, 1.54250806, 1.54250806, 3.37980973,
       4.29720948, 4.29720948, 3.48911338, 3.48911338, 5.66720164,
       5.66720164])

In [55]:
display_sharpness(nbhood, plot_meshvert=False, meshvert_psize=0.01,
                  sharp_vert=points[np.where(values > 1.1)[0]], sharpvert_psize=2 * sampler.resolution_3d,
                  samples=noisy_points, samples_distances=distances,
                  samples_color=0x0000ff, samples_psize=sampler.resolution_3d)

Output()

In [ ]:
import igl

In [ ]:
_, point_face_indexes, _ = \
    igl.point_mesh_squared_distance(noisy_points, nbhood.vertices, nbhood.faces)



In [ ]:
from sharpf.utils.abc_utils import get_adjacent_features_by_bfs_with_depth1, build_surface_patch_graph

In [ ]:

# understand which surface patches are adjacent to which sharp features
# and other surface patches
adjacent_sharp_features, adjacent_surfaces = build_surface_patch_graph(nbhood_features)



In [ ]:
adjacent_sharp_features, adjacent_surfaces

In [ ]:

# compute distance, iterating over points sampled from corresponding surface patches
distances, projections = np.ones(len(points)) * annotator.distance_upper_bound, np.zeros_like(points)



In [ ]:
surface_idx, surface = 3, nbhood_features['surfaces'][3]

In [ ]:

# constrain distance computation to certain sharp features only
adjacent_sharp_indexes = get_adjacent_features_by_bfs_with_depth1(
    surface_idx, adjacent_sharp_features, adjacent_surfaces)


In [ ]:
adjacent_sharp_indexes

In [ ]:
surface_adjacent_features = {
    'curves': [nbhood_features['curves'][idx]
               for idx in np.unique(adjacent_sharp_indexes)]
}


In [ ]:
len(surface_adjacent_features['curves'])

In [ ]:
from sharpf.utils.mesh_utils.indexing import in2d


In [ ]:
point_cloud_indexes = np.where(
    in2d(nbhood.faces[point_face_indexes], surface['face_indices'])
)[0]


In [ ]:
point_cloud_indexes

In [ ]:
# compute distances using parent class AABB method
surface_matching_edges, surface_projections, surface_distances = \
    annotator.compute_aabb_nearest_points(nbhood, surface_adjacent_features, noisy_points[point_cloud_indexes])


In [ ]:
distances[point_cloud_indexes], projections[point_cloud_indexes] = surface_distances, surface_projections


In [ ]:
display_sharpness(nbhood, plot_meshvert=False, meshvert_psize=0.01,
                  sharp_vert=None, sharpvert_psize=2 * sampler.resolution_3d,
                  samples=noisy_points, samples_distances=distances,
                  samples_color=0x0000ff, samples_psize=sampler.resolution_3d)

In [ ]:
def compute_bounded_labels(points, projections, distances=None, max_distance=np.inf, distance_scaler=1.0):
    if distances is None:
        distances = np.linalg.norm(projections - points, axis=1)

    distances = distances / distance_scaler
    # boolean mask marking objects far away from sharp curves
    far_from_sharp = distances > max_distance
    distances[far_from_sharp] = max_distance
    distances = distances.reshape(-1, 1)
    # compute directions for points close to sharp curves
    directions = np.zeros_like(points)
    directions[~far_from_sharp] = projections[~far_from_sharp] - points[~far_from_sharp]
    eps = 1e-6
    directions[~far_from_sharp] /= (np.linalg.norm(directions[~far_from_sharp], axis=1, keepdims=True) + eps)
    return distances, directions

In [ ]:

distances, directions = compute_bounded_labels(
    noisy_points, projections, distances=distances,
    max_distance=annotator.distance_upper_bound)

In [ ]:
display_sharpness(nbhood, plot_meshvert=False, meshvert_psize=0.01,
                  sharp_vert=None, sharpvert_psize=2 * sampler.resolution_3d,
                  samples=noisy_points, samples_distances=distances,
                  samples_color=0x0000ff, samples_psize=sampler.resolution_3d)

In [ ]:
from scipy.spatial import cKDTree


In [ ]:
nn_distances, nn_indexes = cKDTree(noisy_points, leafsize=16).query(noisy_points, k=2, n_jobs=4)


In [ ]:
nn_distances.shape

In [ ]:
values = np.abs(distances[nn_indexes[:, 0]] - distances[nn_indexes[:, 1]]) / np.atleast_2d(nn_distances[:, 1]).T


In [ ]:
ii, jj = np.where(values > 1.1)

In [ ]:
ii

In [ ]:
display_sharpness(nbhood, plot_meshvert=False, meshvert_psize=0.01,
                  sharp_vert=None, sharpvert_psize=2 * sampler.resolution_3d,
                  samples=noisy_points[ii, jj], samples_distances=distances[ii, jj],
                  samples_color=0x0000ff, samples_psize=sampler.resolution_3d)

In [ ]:

    # constrain distance computation to certain sharp features only
    adjacent_sharp_indexes = get_adjacent_features_by_bfs_with_depth1(
        surface_idx, adjacent_sharp_features, adjacent_surfaces)
    surface_adjacent_features = {
        'curves': [features_patch['curves'][idx]
                   for idx in np.unique(adjacent_sharp_indexes)]
    }
    if len(surface_adjacent_features['curves']) == 0:
        continue

    point_cloud_indexes = np.where(
        in2d(mesh_patch.faces[point_face_indexes], surface['face_indices'])
    )[0]
    # point_cloud_indexes = np.where(np.isin(closest_nbhood_vertex_idx, surface['vert_indices']))[0]
    if len(point_cloud_indexes) == 0:
        continue
    # compute distances using parent class AABB method
    surface_matching_edges, surface_projections, surface_distances = \
        self.compute_aabb_nearest_points(mesh_patch, surface_adjacent_features, points[point_cloud_indexes])
    distances[point_cloud_indexes], projections[point_cloud_indexes] = surface_distances, surface_projections

return projections, distances



In [ ]:
nbhood.vertices.shape

In [ ]:
display_sharpness(nbhood, plot_meshvert=False, meshvert_psize=0.01,
                  sharp_vert=None, sharpvert_psize=2 * sampler.resolution_3d,
                  samples=noisy_points, samples_distances=distances,
                  samples_color=0x0000ff, samples_psize=sampler.resolution_3d)

In [ ]:
mean_mmd(points)